In [2]:
import copy
from collections import deque



In [3]:
def read_a_graph(file_name):
    %pwd
    %cd /Users/AM/Documents/_Courses/Algorithms/Misc
    # load a comma-delimited text file into an np matrix
    resultList = {}
    f = open(file_name, 'r')
    for line in f:
        aa = line.rstrip('\n')  # "1.0 \t 2.0 \t 3.0"
        sVals = aa.split('\t')
        sVals_s = sVals[:-1] # remove the last ''
        intVals = list(map(np.int, sVals_s))  # [1.0, 2.0, 3.0]
        resultList[intVals[0]] = intVals[1:]
    
    f.close()
    out = resultList
    return out#np.asarray(resultList, dtype=np.int)  

In [99]:
# Programming assignment 2.1 Programming assignment. MIN CUT
class Graph(object):

    def __init__(self):
        self.vert = [] #list of all verteces
        self.al_d = {} # adjacency list for the graph (self)
        self.al_r = {} # adjacency list for the reversed graph (self)
        self.max_i = 0 # max index of the graph nodes (or verteces)
        self.explr = [] # exploration flags for DFF; 
        self.ft_r = [] # b - finishing time reversed graph
                        # c - finishing time reversed
    def find_all_vert(self, file_name):
        %pwd
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        resultList = []
        f = open(file_name, 'r')
        max_i = self.max_i
        for line in f:
            line_s = line.rstrip('\n')  # "1.0 2.0 3.0"    
            sVals = line_s.split(' ')
            sVals_s = sVals[:-1] # remove the last ''
            intVals = list(map(np.int, sVals_s))  # [1.0, 2.0, 3.0]
            if intVals[0] > max_i: max_i = intVals[0] # looking for a vertex with the max index
            if intVals[1] > max_i: max_i = intVals[1]

            resultList.append(intVals[0])
            resultList.append(intVals[1])
              
        f.close()
        myset = set(resultList)
        self.vert = list(myset)
        self.vert.sort()
        self.max_i = max_i
        self.explr = [False for x in range(self.max_i + 1)]
        self.ft = [0 for x in range(self.max_i + 1)]

    def create_adj_lists(self, file_name):
        %pwd
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        # load a comma-delimited text file into an np matrix
        resultList_d = {x:[] for x in self.vert}
        resultList_r = {x:[] for x in self.vert}

        f = open(file_name, 'r')
        for line in f:
            line_s = line.rstrip('\n')  # "1.0 2.0 3.0"    
            sVals = line_s.split(' ')
            sVals_s = sVals[:-1] # remove the last ''
            intVals = list(map(np.int, sVals_s))  # [1.0, 2.0, 3.0]
            
            resultList_d[intVals[0]].append(intVals[1])
            resultList_r[intVals[1]].append(intVals[0])

        
        f.close()
        self.al_d = resultList_d
        self.al_r = resultList_r

    def dfs_rev(self):
        self.explr = [False for x in range(self.max_i + 1)] # reset explored flags
        self.ft_r = [-1 for x in range(self.max_i + 1)]

        q_dfs = deque([list(self.al_r.keys())[0]]) # add to a dfs queue the first vertex 
        fin_time = 0
        
        while q_dfs:
            i = q_dfs.pop() # take a next node to explore 
            self.explr[i] = True # mark as explored the node i
            self.al_r[i].sort() # make a sorted and reversed list of all the nodes 
            self.al_r[i].reverse()   
            a = self.al_r[i]    
            
            new_vertex_discovered = False
            for ver in a:
                if self.explr[ver] == False: # if we find a new vertex
                    q_dfs.append(ver)
                    self.explr[ver] = True 
                    new_vertex_discovered = True
            if new_vertex_discovered == False:
                fin_time += 1
                self.ft_r[i] = fin_time

        
    
#    def scc
    
    
    
  

In [100]:
gr = Graph()
gr.find_all_vert('text4.txt')


/Users/AM/Documents/_Courses/Algorithms/misc


In [101]:
gr.create_adj_lists('text4.txt')

/Users/AM/Documents/_Courses/Algorithms/misc


In [14]:
# Programming assignment 2.2 Programming assignment. SHORTEST PATH

from ast import literal_eval


class DijkstraPathFinder:

    def __init__(self, input_file):
        self.graph = {}
        #%pwd
        #%cd /Users/AM/Documents/_Courses/Algorithms/Misc
        with open(input_file) as file:
            for line in file:
                line_content = line.split()
                self.graph[int(line_content[0])] = [literal_eval(edge) for edge in line_content[1:]]
        self._source_vertex = next(iter(self.graph.keys()))

    def compute_shortest_paths(self, source=None):
        if source is None:
            source = self._source_vertex
        shortest_paths = {}
        visited = set()
        for vertex in self.graph.keys():
            shortest_paths[vertex] = (9999999999, [])
        shortest_paths[source] = (0, [])
        visited.add(source)
        while set(self.graph.keys() - visited):
            source, min_edge = -1, ()
            for vertex in visited:
                for edge in self.graph[vertex]:
                    if edge[0] in visited:
                        continue
                    if not min_edge or shortest_paths[vertex][0] + edge[1] < min_edge[1]:
                        min_edge = (edge[0], shortest_paths[vertex][0] + edge[1])
                        source = vertex
            shortest_paths[min_edge[0]] = (min_edge[1], shortest_paths[source][1] + [min_edge[0]])
            visited.add(min_edge[0])
        return shortest_paths

if __name__ == '__main__':
    path_finder = DijkstraPathFinder('text5.txt')
    paths = path_finder.compute_shortest_paths()
    actual = {vertex: distance[0] for (vertex, distance) in paths.items()}
    print(actual[7])
    print(actual[37])
    print(actual[59])
    print(actual[82])
    print(actual[99])
    print(actual[115])
    print(actual[133])
    print(actual[165])
    print(actual[188])
    print(actual[197])

2599
2610
2947
2052
2367
2399
2029
2442
2505
3068


In [102]:
print('max vert index', gr.max_i, 'number of vert', len(gr.vert))

max vert index 875714 number of vert 875714


In [103]:
gr.dfs_rev()